In [1]:
CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

### Detect images using mobile net

In [2]:
from torchvision.models import detection
import numpy as np
import argparse
import torch
import cv2


In [3]:
thresh = 0.5

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#CLASSES = open("coco-labels.txt").read().strip().split('\n')
COLORS = np.random.uniform(0, 255, size = (len( CLASSES ), 3))

model = detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True, progress = True,
                                                        num_classes = len(CLASSES), pretrained_backbone=True).to(DEVICE)

model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(320,), max_size=640, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequen

In [4]:
thresh = 50
path = r"C:\Users\Hugo-Tech-482\Downloads\HugoPegs\AVA_7807-1-copy-1-scaled-e1603204431735 (1).jpg"


image = cv2.imread(path)
orig = image.copy()


# Switch channels
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.transpose((2, 0, 1))

# Process and normalize
image = np.expand_dims(image, axis = 0) # creates a single batch of input image
image = image / 255.0 # scales
image = torch.FloatTensor(image)

# Image to device
image = image.to(DEVICE)
pred = model(image)[0]

for obj in range(0, len(pred['boxes'])):
    conf = pred['scores'][obj] * 100
    
    if conf > thresh:
        idx = int(pred["labels"][obj])
        bbox = pred["boxes"][obj].detach().cpu().numpy()
        (x1, y1, x2, y2) = bbox.astype('int')
        
        label = f"{CLASSES[idx]}: {conf:.2f}%"
        print(f"[INFO] {label}")

        cv2.rectangle(orig, (x1, y1), (x2, y2), COLORS[idx], 2)
        y = y1 -15 if y1 - 15 < 15 else y1 + 15
        cv2.putText(orig, label, (x1, y), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
cv2.imshow('Output', orig)
cv2.waitKey(0)

cv2.destroyAllWindows()

C:\Users\Hugo-Tech-482\anaconda3\envs\torch_session\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[INFO] person: 99.98%
[INFO] laptop: 99.50%
[INFO] person: 99.29%
[INFO] person: 87.06%
[INFO] tie: 55.61%
